In [1]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
import pandas as pd

/home/nivag/2023-Linux/.python-3.10/lib/python3.10/site-packages/statsforecast/core.py:25: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
average_RRC = pd.read_csv('sample-folder/IOH-Active User-4.csv')

In [3]:
list1 = average_RRC.columns[0:]
result = pd.DataFrame()

In [4]:
for x in range(1, len(average_RRC.columns), 1):

    sitelist = average_RRC[['SECTOR-ID']]
    duration_week = pd.DataFrame(data={'date': [list1[x]] * len(average_RRC.index)})
    content = pd.DataFrame(average_RRC.iloc[:,x]).reset_index(drop=True)
    content.rename({content.columns[0]:'RRC'}, axis=1, inplace=True)

    concat = pd.concat([sitelist, duration_week, content], axis=1)
    result = pd.concat([result, concat], axis=0)

In [5]:
result.reset_index(inplace=True, drop=True)
result['Year'] = result['date'].str.split('-W').str[0]
result['WeekNumber'] = result['date'].str.split('-W').str[1].astype(int)

result['StartOfWeek'] = pd.to_datetime(result['Year'], format='%Y') + pd.to_timedelta((result['WeekNumber'] - 1) * 7 + 1, unit='D')
result.drop(columns=['Year', 'WeekNumber'], inplace=True)

result.rename(columns={'RRC': 'y', 'StartOfWeek':'ds', 'SECTOR-ID':'unique_id'}, inplace=True)

In [6]:
# query = result[(result['unique_id'] == 'Grand Total') & (~result['y'].isnull())] [['ds', 'y' , 'unique_id']]
# query.reset_index(inplace=True, drop=True)

# test = query[['unique_id', 'ds', 'y']]

# sf = StatsForecast(
#     models = [AutoARIMA(season_length = 13)],
#     freq = 'W'
# )

# sf.fit(test)

# forecast = sf.forecast(h=71, level=[50], fitted=True) 

In [7]:
concatenate = pd.DataFrame()

for x in result['unique_id'].unique().tolist():
    query = result[(result['unique_id'] == x) & (~result['y'].isnull())] [['ds', 'y', 'unique_id']]

    test = query
    sf = StatsForecast(models = [AutoARIMA(season_length = 13)], freq = 'W')
    sf.fit(test)

    forecast = sf.predict(h=72, level=[50]) 

    forecast.rename(columns={'AutoARIMA-hi-50': 'y'}, inplace=True)
    forecast.drop(columns=['AutoARIMA-lo-50', 'AutoARIMA'], inplace=True)
    forecast = pd.concat([query, forecast], axis=0)
    forecast.reset_index(inplace=True)

    combine = pd.concat([forecast, pd.DataFrame({'SECTOR-ID':[x]*len(forecast.index)})], axis=1)
    concatenate = pd.concat([concatenate, combine], ignore_index=True, axis=0)

In [ ]:
concatenate.drop(columns=['unique_id', 'index'], inplace=True)

In [ ]:
concatenate.to_excel('result/arima-activeuser-sectors-1.xlsx', index=False)